In [27]:
## run LASSO on the entropy of orignal bold signals.

import pandas as pd
import numpy as np
import os
import re
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score

input_dir  = 'hcp_out04_bold_entropy_0back/'
input_dir2 = 'hcp_out04_bold_entropy_2back/'

output_dir = 'hcp_out05_lasso_bold_entropy/'


In [28]:

f1 = input_dir  + '/out04_bold_entropy.csv'
f2 = input_dir2 + '/out04_bold_entropy.csv'

all_result = []

data = pd.read_csv(f1, index_col = None)
data2= pd.read_csv(f2, index_col = None)

num1, num2 = data.shape[0], data2.shape[0]
X = pd.concat([data, data2], axis = 0).values
y = np.hstack((np.ones(num1), np.zeros(num2)))

nan_row = np.isnan(X).any(axis = 1)
X = X[~nan_row,:]
y = y[~nan_row]

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
C_values = np.logspace(-1, 2, 10)

# select data with ad/lmci and cn:    
# build lassoCV that tune parameters with inner cv:
reg = LogisticRegressionCV(cv=5, random_state=0, Cs=C_values, n_jobs = 6,
                           penalty = 'l1', solver='liblinear', 
                           refit = False).fit(X, y)
# reg = LassoCV(cv=5, random_state=0, normalize = True, tol = .001).fit(X[idx_ad_cn,:], y[idx_ad_cn])
# run outer cv:
cv_result = cross_val_score(reg, X, y, cv=5)
print('cross validaton result, mean %3f, std: %3f' % (cv_result.mean(), cv_result.std()))
result_accuracy = list(cv_result)
result_coefs = list(reg.coef_.reshape(-1))
    
print('finished')


cross validaton result, mean 0.560982, std: 0.021913
finished


In [29]:
cv_result

array([0.5859375 , 0.55208333, 0.58072917, 0.5613577 , 0.52480418])

In [30]:
## save results:
## save all accuracy results to one file (as we only have 1 metric for each comparision)
## udpates on May-10-2022.

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

res = pd.DataFrame(result_accuracy)
res.index = ['cv' + str(i) for i in range(1, 6)]
res.columns = ['0back vs 2back']

res.to_csv(output_dir + '/lasso_accuracy.csv')


res = pd.DataFrame(result_coefs)
res.index = ['roi' + str(i) for i in range(1, 269)]
res.columns = ['0back vs 2back']

res.to_csv(output_dir + '/lasso_coefs.csv')


In [31]:
data
# result_accuracy

,0,1,2,3,4,5,6,7,8,9,...,258,259,260,261,262,263,264,265,266,267
0,3.622136,3.545294,3.685286,3.572972,3.476611,3.503074,3.573438,3.611903,3.468661,3.543496,...,3.552309,3.432371,3.678623,3.587482,3.697084,3.562202,3.565025,3.679941,3.393590,3.617382
1,3.608763,3.510536,3.559808,3.618175,3.493226,3.467707,3.585863,3.668670,3.745451,3.663003,...,3.605472,3.319853,3.502183,3.523347,3.540971,3.552881,3.553031,3.556390,3.493102,3.553953
2,3.499441,3.568800,3.477975,3.700670,3.489683,3.705064,3.371010,3.425578,3.582555,3.529946,...,3.416888,3.480133,3.515132,3.656697,3.534594,3.337156,3.569449,3.590504,3.528212,3.569741
3,3.680859,3.554265,3.551559,3.690765,3.591375,3.584108,3.659044,3.574884,3.550833,3.418602,...,3.592649,3.497868,3.458331,3.519872,3.507983,3.615940,3.465945,3.470553,3.320215,3.583718
4,3.427618,3.492409,3.554155,3.432357,3.335808,3.448062,3.536174,3.479205,3.390578,3.393487,...,3.504613,3.648162,3.644509,3.532894,3.405198,3.526846,3.750560,3.619977,3.591187,3.375355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,3.443259,3.325771,3.441832,3.286735,3.565334,3.585986,3.386055,3.441678,3.612787,3.664243,...,3.331305,3.573428,3.374708,3.672317,3.561442,3.691571,3.162114,3.349271,3.555122,3.374874
956,3.425604,3.560712,3.540413,3.529791,3.541095,3.652616,3.720284,3.701790,3.543692,3.316547,...,3.585556,3.638035,3.662879,3.330699,3.469064,3.589308,3.586064,3.651463,3.517972,3.680546
957,3.395916,3.330595,3.462499,3.582508,3.588034,3.185319,3.428341,3.465436,3.569774,3.599681,...,3.492634,3.562161,3.583314,3.573832,3.406364,3.560134,3.651686,3.525865,3.464345,3.598141
958,3.729676,3.542134,3.463292,3.601950,3.505693,3.505018,3.636044,3.640471,3.608548,3.486049,...,3.666213,3.498829,3.632612,3.575566,3.425376,3.585472,3.615961,3.466304,3.405669,3.563747


In [32]:
metrics

NameError: name 'metrics' is not defined